# **Eksploracja danych**

## **Projek 2: Klasyfikacja przy użyciu KNN**  
## Krzysztof Stawarz

In [ ]:
GlassData = cell2mat(struct2cell(load('ed-p02.mat', 'GlassData')));

In [ ]:
GlassClasses = cell2mat(struct2cell(load('ed-p02.mat', 'GlassClasses')));

In [ ]:
TestData = cell2mat(struct2cell(load('ed-p02.mat', 'TestData')));

### **Rozwiązanie typu: od zera**

*Piszemy wszystko od początku (pomimo, że ktoś mądry kiedyś za nas już to napisał, ale to nieistotne!!!)*

#### **Krok 1.**

*Dzielimy obecny dataset na zbiór do nauki ( `train_data` ) i przetestowania rezutlatów ( `test_data` ).  
Robimy to też z labelami - `train_labels` i `test_labels`.*  
  
*Ja preferuję podział 3:1 dla testowego, ale można sobie wybrać dowolny. Raczej nie przekracza się 50%/50%.*

In [ ]:
train_frac = .75;

In [ ]:
%  wektor decyzyjny z losowymi wartosciami, to dzieki niemu decydujemy czy wiersz dopisac do sekcji treningowej czy testowej
decision_vect = rand(size(GlassData, 1), 1);

train_data = GlassData(decision_vect <= train_frac, :);
train_labels = GlassClasses(decision_vect <= train_frac, :);

test_data = GlassData(decision_vect > train_frac, :);
test_labels = GlassClasses(decision_vect > train_frac, :);

#### **Krok 2.**

*Wprowadzamy kilka zmiennych służących nam za skróty, żeby nie pisać kilka razy tego samego*

In [ ]:
train_rows_n = size(train_data, 1)
train_cols_n = size(train_data, 2)

test_rows_n = size(test_data, 1)

#### **Krok 3.**

*Zaczynamy od rzeczy trywialnych. Napiszmy pętlę `for`, zliczającą odległość (w metryce taksówkowej) parametrów z pierwszego wiersza zbioru testowego i treningowego:*

$d(x, y) = \sum\limits_{i=0}^{n}|x_1 - y_1| + |x_2 - y_2| + ... + |x_n - y_n|$

In [ ]:
distance = 0;
for i = 1:train_cols_n
    distance = distance + abs(train_data(1, i) - test_data(1, i));
end

distance

#### **Krok 4.**

*Potrafiąc policzyć dystans jednego wiersza testowego do jednego treningowego, możemy teraz policzyć dystanse jednego wiersza testowego do **każego** wiersza treningowego (zapisując wyniki z zmiennej `distances`:*

In [ ]:
distances = [];

    for j=1:train_rows_n
    
        distance = 0;
        for i = 1:train_cols_n
            distance = distance + abs(train_data(j, i) - test_data(1, i));
        end
        
        distances = [distances, distance];

    end

% dystanse to wektor poziomy, stransponuje go i skrócę do pierwszych 5-ciu wyników w celu wyświetlenia
% aby nie zajmował niepotrzebnie miejsca (bedzie mial +/- 126 wierszy)
distances(:, 1:5)'

*Musimy się na tym etapie stety bądź nie trochę ubrudzić.*  
1. *Będziemy chcieli zmapować dystanse do odpowiednich wierszy jako pary {dystans: label_wiersza}. Użyjemy do tego celu słownika, a właściwie funkcji `sorted_dict`, napisanej w osobnym pliku w tym folderze. Funkcja ta zwraca słownik takich par, posortowany rosnąco według kluczy (dystansów), co będzie przydatne w dalszym etapie rozwiązania*  

In [ ]:
d = sorted_dict(distances', train_labels)

*Posiadając taki słownik, możemy użyć już algorytmu __KNN__ - możemy sklasyfikować obserwację do jednej z sześciu klas.*

2. *Aplikujemy algorytm __KNN__, co oznacza, że będziemy patrzeć na __k__ najbliższych sąsiadów, czyli __k__ najmniejszych dystansów, oraz na labele wierszy, do których się odnoszą (stąd użycie słownika, chcemy śledzić obydwie wartości naraz i ich nie pogubić)*  
*Do wyznaczania współczynnika __k__ przejdziemy w ostatnim kroku, na razie przyjmijmy __k=10__ na potrzeby prezentacji pomysłu.*  
*Funkcja `sorted_dict` ma zaimplementowane użycie trzeciego argumentu jako współczynnika __k__ - uzystany w ten sposób słownik będzie posiadał __k__ pierwszych par.*

In [ ]:
k = 10;

d = sorted_dict(distances', train_labels, k)

3. *Gdy uzyskamy słownik z __k__ najmniejszymi dystansami, musimy znaleźć najczęściej występujący w nim label - to będzie nasza __predykacja, strzał, przewidywanie lub klasyfikacja__, jak zwał tak zwał*  
*Do znaleziena najczęściej występującej wartości (labelu) w słowniku służy funkcja `most_frequent_value`, napisana w osobnym pliku w tym folderze.*

In [ ]:
predicted_label = most_frequent_value(d)

*Powyższa odpowiedź oznacza, że nasz zaimplementowany algorytm na podstawie danych jakie otrzymał (pamiętajmy __k__ = 10) uważa, że pierwszy wiersz w zbiorze testowym powinien należeć do klasy nr 1.*  
*Z racji, że zbiór testowy wyodrębniliśmy z naszego pierwotnego datasetu, możemy sprawdzić dokładność tej predykcji w niezwykle binarny sposób - przyrównać predykcję algorytmu do faktycznie przydzielonej do obserwacji klasy:*

In [ ]:
predicted_label == test_labels(1, :)

*Hurra! Udało nam się napisać algorytm KNN dla jednego wiersza!*  
  
#### __Krok 5.__  
  

*Teraz zamknijmy go w pętli, aby przeszedł po każdym wierszu ze zbioru do testu. W dodatku będziemy mogli w ten sposob policzyć jego dokładność, śledząc stosunek udanych predykcji do wszystkich obrotów pętli `for`:*

In [ ]:
all_guesses = 0;
correct_guesses = 0;

for test_row=1:test_rows_n   
    all_guesses = all_guesses + 1;
    
    distances = [];

    for j=1:train_rows_n

        distance = 0;
        for i = 1:train_cols_n
            distance = distance + abs(train_data(j, i) - test_data(test_row, i));
        end

        distances = [distances, distance];

    end
    
     d = sorted_dict(distances', train_labels, k);
     predicted_label = most_frequent_value(d);

     if predicted_label == test_labels(test_row, :)
        correct_guesses = correct_guesses + 1;
     end
    
end

accuracy = correct_guesses / all_guesses

*Taka jest dokładność naszego algorytmu przy takim losowym podziale na testówke i treningówę i, co ważne, przy parametrze __k__=10*

#### __Krok 6.__

*Jak znajdziemy najbardziej optymalny współczynnik __k__? My użyjemy algorytmu nazywającego się __brute force__. Przeitereujemy po wszystkich możliwych wartościach __k__ i znajdziemy ten z największą dokładnością!*  
__UWAGA__ : zabezpieczymy się przed losowością kilku pierwszych wartości __k__ (dokładnie to pierwszych pięciou) przy użyciu `if`!!!

In [ ]:
accuracy_arr = [];

for k = 1:train_rows_n   

    all_guesses = 0;
    correct_guesses = 0;

    for test_row = 1:test_rows_n   
        all_guesses = all_guesses + 1;

        distances = [];

        for j = 1:train_rows_n

            distance = 0;
            for i = 1:train_cols_n
                distance = distance + abs(train_data(j, i) - test_data(test_row, i));
            end

            distances = [distances, distance];

        end

         d = sorted_dict(distances', train_labels, k);
         predicted_label = most_frequent_value(d);

         if predicted_label == test_labels(test_row, :)
            correct_guesses = correct_guesses + 1;
         end

    end

    accuracy = correct_guesses / all_guesses;
    
    if k <= 5
        accuracy = 0;
    end
    
    accuracy_arr = [accuracy_arr, accuracy];
end

*Algorytm napisany, poniżej wizualizacja podsumowująca wszystkie współczynniki __k__ wraz z ich dokładnościami:*

In [ ]:
x = 1:train_rows_n;
y = accuracy_arr;

plot(x, y, 'b-');
xlim([1 train_rows_n]);
ylim([min(accuracy_arr(:, 6:end)) max(accuracy_arr)]);

hold on;

k = find(accuracy_arr == max(accuracy_arr), 1)
k_acc = max(accuracy_arr)

plot(k, k_acc, 'rsquare');

*W ten oto sposób napisaliśmy prosty algorytm __KNN__ od zera oraz przy okazji znaleźliśmy najbardziej optymalny współczynnik __k__ dla naszego datasetu.*  

#### __Krok 7.__

*Teraz należy tylko użyć algorytmu i spredykować klasy dla wierszy ze zmiennej `TestData`:*

In [ ]:
predicts = [];

for test_row = 1:size(TestData, 1)  

        distances = [];

        for j = 1:train_rows_n

            distance = 0;
            for i = 1:train_cols_n
                distance = distance + abs(train_data(j, i) - TestData(test_row, i));
            end

            distances = [distances, distance];

        end

         d = sorted_dict(distances', train_labels, k);
         predicted_label = most_frequent_value(d);
         
         predicts = [predicts predicted_label];
end


ans = dictionary(1:size(TestData, 1), predicts)

*Krzysztof Stawarz*  
*Kraków, 14.03.2023*